In [163]:
import torch
from torch import nn, optim, autograd as grad
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import transforms, datasets 
import matplotlib.pyplot as plt

This code classifies 10 different classes of images

In [164]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.convL1 = nn.Conv2d(in_channels=3,out_channels=128, 
                                kernel_size=(3,3), stride= (1,1),padding=2)
        nn.init.kaiming_normal(self.convL1.weight)
        self.convL2 = nn.Conv2d(in_channels=128,out_channels=64, 
                                kernel_size=(3,3), stride= (1,1),padding=2)
        nn.init.kaiming_normal(self.convL2.weight)
        self.convL3 = nn.Conv2d(in_channels=64,out_channels=32, 
                                kernel_size=(2,2), stride= (1,1),padding=2)
        nn.init.kaiming_normal(self.convL1.weight)


        self.linL4 = nn.Linear(2592, 1000, bias=True)
        self.linL5 = nn.Linear(1000, 10, bias=True)
        nn.init.kaiming_uniform_(self.linL4.weight)
        nn.init.kaiming_uniform_(self.linL5.weight)
        self.pool3 = nn.MaxPool2d((3,3), (3,3))
        self.pool2 = nn.MaxPool2d((2,2), (2,2))

        self.dropout = nn.Dropout()
        self.relu = nn.ReLU()  
        self.Batch_norm = nn.BatchNorm1d(num_features=32)
        self.Lrelu = nn.LeakyReLU()
        self.Softplus = nn.Softplus()
        self.Softmax = nn.Softmax()

    def forward(self, x):
        x=self.Batch_norm(x)
        x=self.Lrelu(self.convL1(x))
        x=self.pool3(x)
        x=self.relu(self.convL2(x))
        x=self.pool2(x)
        x=self.relu(self.convL3(x))

        x=torch.flatten(x)
        x=self.Lrelu(self.linL4(x))
        x=self.dropout(x)
        x=self.Softmax(self.linL5(x))
        return x

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

Additional Functions

In [165]:
def ret_shape(training_set)->str:
    ret_string = ""
    for idx, data in enumerate(training_set):
            datas = data[0]
            labels = data[1]
            ret_string+="{}\n".format(datas.shape)
            ret_string+="Labels:{}\n".format(labels)
            ret_string+="Labels shape:{}\n".format(len(labels))
            ret_string+="Labels[0] shape:{}\n".format(labels[0].shape)
            break
    return ret_string


def max_index(max_tnsr):
    max_tnsr = torch.tensor(max_tnsr)
    max_tnsr = max_tnsr.tolist()
    max_num = 0
    for num in max_tnsr:
        if max_num < num:
            max_num = num
    return max_tnsr.index(max_num)


def val_func(convNet, cnn_num,valDS): # Selects the best network
                                                #   out of n networks
    cnns_loss = [0]*cnn_num # represents the 10 cnns
    print("\n| Starting validation set run |\n")
    for val_sample in valDS:
        for j in range(cnn_num):
            prediction = convNet[j].forward(torch.squeeze(val_sample[0]))
            loss_func = nn.CrossEntropyLoss()
            loss = loss_func(prediction, val_sample[1])
            cnns_loss[j]+=loss # Adds the loss of the
    cnns_loss = [j*-1 for j in cnns_loss]
    return convNet[max_index(cnns_loss)]# picks the cnn with the lowest loss


def train_one_epoch(Cnn, FILE_PATH, train_ds, optimizer):
    iter = 0

    for sample in train_ds:
        targetY = torch.nn.functional.one_hot(
        torch.tensor(sample[1]), num_classes=10)
        targetY = torch.tensor(targetY, dtype=torch.float32, requires_grad=True)
        targetY = torch.squeeze(targetY)
            
        #optimizer = optim.Adam(params=Cnn.parameters(), weight_decay=0.1
        #                      , lr=1e-4, betas=[0.09, 0.0999])
        
        pred = Cnn.forward(torch.squeeze(sample[0]))
        loss_func = nn.CrossEntropyLoss()
        # ---BackPropagation---
        loss = loss_func(pred, targetY)
        loss.backward()
        optimizer.step()

        if iter%100==0:
            torch.save(Cnn.cpu(), FILE_PATH)
            grad_tmp = Cnn.linL5.weight.grad
            print(iter," | ",grad_tmp.sum()/len(grad_tmp))

        optimizer.zero_grad()
        iter+=1


Train Cell:

In [166]:
FILE_PATH = "model.pth"
plot_lst = []
cnn_num = 10


def train_net(cnn, train_ds, pretrain_ds, val_ds, dev):
    lrs = [num / int(1e4) for num in range(1, 11)]# The learning rates of
                                                #the 10 different CNNs
    momentums = [num/10 + 0.09 for num in range(0,10)]
    momentums = momentums[4:] + momentums[:4]# Shift from the middle
    
    #optimizer = optim.Adam(params=cnn.parameters(), weight_decay=0.01
    #                    , lr=1e-4, betas=[0.6, 0.6])
    

    # Multiple pretraining part with a minibatch (10,000)
    for i in range(len(cnn)):
        optimizer = optim.SGD(params=cnn[i].parameters(), lr=lrs[i],
                            momentum=momentums[i], weight_decay=0.0001)
        train_one_epoch(cnn[i], FILE_PATH, train_ds=pretrain_ds, optimizer=optimizer)

    optimizer = optim.SGD(params=cnn.parameters(), lr=8e-4,
                            momentum=0.699,
                            weight_decay=0.0001)

    


In [ ]:
if __name__ == '__main__':

    # Accessing processing unit
    net = [0]*cnn_num
    if torch.cuda.is_available() : device = "cuda:0"
    else : device = "cpu"
    device = torch.device(device)
    for i in range(cnn_num):
        net[i]=CNN().to(device)

    # Loading the dataset

    train_data = torchvision.datasets.CIFAR10(
        root= "C:/Users/orian/OneDrive/שולחן העבודה/My Coding Files"
        ,train=True, transform=transforms.Compose([transforms.ToTensor()
                        #,transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                        ])
        #, target_transform=transforms.Compose([transforms.ToTensor()])
    )

    val_split = 500
    pretrain_ds, tmp = random_split(train_data, 
                [val_split, len(train_data)-val_split])
    pretrain_ds = DataLoader(pretrain_ds, shuffle=True)
    train_data = DataLoader(train_data, shuffle=True)

    test_ds = torchvision.datasets.CIFAR10(
            root= "C:/Users/orian/OneDrive/שולחן העבודה/My Coding Files"
            ,train=False, transform=transforms.Compose([transforms.ToTensor()]), 
        )
    val_ratio = 0.01
    val_split = int(val_ratio*len(test_ds))
    val_data, test_data = random_split(
        test_ds, [val_split, len(test_ds)-val_split]
    )
    
    train_net(net, train_data, pretrain_ds, val_data, dev=device)


Test Cell:

In [ ]:
FILE = "model.pth"
model = torch.load(FILE)
model.eval()

image_lst = [0]*10
right_pred = 0
for i in range(len(test_data)):
    if max_index(model.forward(test_data[i][0]))==test_data[i][1]:
        right_pred+=1
    image_lst[max_index(model.forward(test_data[i][0]))]+=1
    
print("model's accuracy:",right_pred/100,"%")
print(image_lst)
plt.plot(plot_lst)
plt.show()